<a href="https://colab.research.google.com/github/defneyucesir/gametheory/blob/main/Sentiment_Analysis_and_Defining_Utilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install spacy
!pip install scattertext
!pip install tika
!pip install spacytextblob
!python -m spacy download en_core_web_sm
import spacy
import json
import pylab
from IPython.core.display import display, HTML
import nltk
from tika import parser
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from spacytextblob.spacytextblob import SpacyTextBlob

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10., 8.)
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

import re
def process_urls(csv):
    try:
        df = pd.read_csv(csv)
    except:
        print(f"Error reading the file")
        return None

#Error handling
    if not all(col in df.columns for col in ['Country', 'URL']):
        print("Error: 'country' and 'url' columns are required.")
        return None

    processed_urls = set()
    results = []

    for index, row in df.iterrows():
        country = row['Country']
        url = row['URL']

        if url in processed_urls:
            continue

#Using TIKA parser
        try:
            raw = parser.from_file(url)
            text = raw['content']

#Data cleaning
            if text:
                text = " ".join(text.split())
                text = re.sub(r"[^a-zA-Z ]+", "", text)
                text = text.lower()

#Calculate polarity and subjectivity
            if text:
                doc = nlp(text)
                polarity = doc._.blob.polarity
                subjectivity = doc._.blob.subjectivity

#Classify sentiment based on polarity scores
                if polarity > 0.05 :
                    sentiment_score_classification = "Positive"
                elif polarity < -0.05:
                    sentiment_score_classification = "Negative"
                else:
                    sentiment_score_classification = "Neutral"

                results.append([country, url, polarity, subjectivity, sentiment_score_classification])
                processed_urls.add(url)

#Error handling
        except:
            print(f"Error processing URL")
            continue

    new_df = pd.DataFrame(results, columns=['country', 'url', 'polarity', 'subjectivity', 'sentiment_score_classification'])
    return new_df

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():

  processed_data = process_urls(fn)

processed_data

In [ ]:
average_sentiment = processed_data['polarity'].mean()
print(f"Average Sentiment Score: {average_sentiment}")